In [1]:
import sys
sys.path.append('../xrun')

In [2]:
import re

from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

from IPython.core.display import HTML
from sklearn.metrics import pairwise_distances

from xrun.data.run_info import RunInfo

In [3]:
def load_cost_from_file(file_path: Path):
    if file_path.exists():
            with open(file_path, "r") as f:
                return float(f.read())
    else:
        # print(f"Warning: {file_path} not found!")
        pass
    return None

def get_algorithm_name(run_info):
    if run_info.algorithm == "basic-clustering":
        return "StreamKM++ (buggy)"
    elif run_info.algorithm == "sensitivity-sampling":
        return "Sensitivity Sampling"
    elif run_info.algorithm == "group-sampling":
        return "Group Sampling"
    elif run_info.algorithm == "bico":
        return "BICO"
    elif run_info.algorithm == "stream-km++":
        return "StreamKM++"
    elif run_info.algorithm == "ray-maker":
        return "Ray Maker"
    return "Unknown"

def get_dataset_print_name(run_info):
    d = {
        "census": "Census",
        "censuslowd": "Census+PCA",
        "covertype": "Covertype",
        "covertypelowd": "Covertype+PCA",
        "tower": "Tower",
        "hardinstanceb1": "Benchmark",
        "hardinstanceb15": "Benchmark-1.5",
        "hardinstanceb2": "Benchmark-2.0",
        "oldhardinstanceb1": "Old Benchmark",
        "oldhardinstanceb2": "Old Benchmark-2.0",
        "caltech101": "Caltech",
        "caltech101lowd": "Caltech+PCA",
        "oldcaltech101": "Old Caltech",
        "nytimes100d": "NYTimes",
        "nytimespcalowd": "NYTimes+PCA",
    }
    if run_info.dataset in d:
        return d[run_info.dataset]
    return run_info.dataset


def compute_distortion(real_cost, coreset_cost):
    if real_cost is not None and coreset_cost is not None:
        return max(float(real_cost/coreset_cost), float(coreset_cost/real_cost))
    return None


def get_costs(file_paths):
    costs = []
    for index, run_file_path in enumerate(file_paths):
        run_info = RunInfo.load_json(run_file_path)
        if "hardinstance" in run_info.dataset and run_info.k > 40:
            continue
        if run_info.algorithm == "stream-km++" and run_info.m > 200*run_info.k:
            continue
        real_cost = load_cost_from_file(run_file_path.parent / "real_cost.txt")
        coreset_cost = load_cost_from_file(run_file_path.parent / "coreset_cost.txt")
        real_cost_synthetic = load_cost_from_file(run_file_path.parent / "real_cost_synthetic.txt")
        coreset_cost_synthetic = load_cost_from_file(run_file_path.parent / "coreset_cost_synthetic.txt")
        real_cost_synthetic2 = load_cost_from_file(run_file_path.parent / "real_cost_convexsynthetic.txt")
        coreset_cost_synthetic2 = load_cost_from_file(run_file_path.parent / "coreset_cost_convexsynthetic.txt")
        distortion = compute_distortion(real_cost, coreset_cost)
        distortion_synthetic = compute_distortion(real_cost_synthetic, coreset_cost_synthetic)
        distortion_synthetic2 = compute_distortion(real_cost_synthetic2, coreset_cost_synthetic2)
        costs.append({
            "dataset": get_dataset_print_name(run_info),
            "algorithm": get_algorithm_name(run_info),
            "k": run_info.k,
            "coreset_size": run_info.m,
            "coreset_size_factor": int(run_info.m / run_info.k),
            "running_time": int(run_info.duration_secs),
            "real_cost": real_cost,
            "coreset_cost": coreset_cost,
            "distortion": distortion,
            "real_cost_synthetic": real_cost_synthetic,
            "coreset_cost_synthetic": coreset_cost_synthetic,
            "distortion_synthetic": distortion_synthetic,
            "distortion_synthetic2": distortion_synthetic2,
            "run_file_path": str(run_file_path),
        })
    df_data = pd.DataFrame(costs)
    return df_data.groupby(["dataset", "algorithm", "k", "coreset_size"], as_index=False).head(10)
    #return df_data

def aggregate_costs(df_costs: pd.DataFrame):
    group_columns = ["dataset", "algorithm", "k", "coreset_size", "coreset_size_factor"]
    df_top_k = df_costs.groupby(group_columns, as_index=False).head(10)
    df_aggr_costs = df_top_k.groupby(group_columns, as_index=False).agg(
        experiment_count=("algorithm", "count"),
        real_cost_mean=("real_cost", "mean"),
        real_cost_std=("real_cost", "std"),
        coreset_cost_mean=("coreset_cost", "mean"),
        coreset_cost_std=("coreset_cost", "std"),
        running_time_mean=("running_time", "mean"),
        running_time_std=("running_time", "std"),
        distortion_mean=("distortion", "mean"),
        distortion_median=("distortion", "median"),
        distortion_std=("distortion", "std"),
        distortion_max=("distortion", "max"),
        distortion_synthetic_mean=("distortion_synthetic", "mean"),
        distortion_synthetic_std=("distortion_synthetic", "std"),
        distortion_synthetic2_mean=("distortion_synthetic2", "mean"),
        distortion_synthetic2_std=("distortion_synthetic2", "std"),
    )
    df_aggr_costs["running_time"] = pd.to_timedelta(df_aggr_costs.running_time_mean, unit='s')
    df_aggr_costs["running_time_formatted"] = df_aggr_costs["running_time"].map(
        lambda x: f"{x.components.hours:02d}h {x.components.minutes:02d}m {x.components.seconds:02d}s"
    )
    return df_aggr_costs


def display_results_for(df_aggr_costs: pd.DataFrame, dataset_name: str, show_counts: bool=True, show_running_times:bool=True, show_costs: bool=True):
    pd.set_option('display.float_format', '{:.2e}'.format)
    
    display(HTML(f"<h2 style='border-bottom:solid 1px Black;padding-bottom:5px;'>Results for {dataset_name}</h2>"))
    df_filtered = df_aggr_costs[df_aggr_costs.dataset == dataset_name]
    
    html_str = ""
    
    if show_counts:
        df_counts = pd.pivot_table(df_filtered, values="experiment_count", index=["algorithm"], columns=["k"])
        df_counts = df_counts.rename_axis(None, axis=0).rename_axis(None, axis=1)
        html_str += f'<div style="border:solid 1px White; width:300px;float:left;">'
        html_str += f"<h4 style='text-align:center;'>Experiment Counts</h4>"
        html_str += df_counts.style.format(precision=0).to_html()
        html_str += "</div>"
    
    if show_running_times:
        df_run_times = pd.pivot_table(df_filtered, values="running_time_formatted", index=["algorithm"], columns=["k"], aggfunc=lambda x: x)
        df_run_times = df_run_times.rename_axis(None, axis=0).rename_axis(None, axis=1)

        html_str += f'<div style="border:solid 1px White; float:left;">'
        html_str += f"<h4 style='text-align:center;'>Average Running Times</h4>"
        html_str += df_run_times.to_html()
        html_str += "</div>"
        
    display(HTML(html_str))

    if show_costs:
        df_real_costs = pd.pivot_table(df_filtered, values="real_cost_mean", index=["algorithm"], columns=["k"])
        df_real_costs = df_real_costs.rename_axis(None, axis=0).rename_axis(None, axis=1)
        
        df_coreset_costs = pd.pivot_table(df_filtered, values="coreset_cost_mean", index=["algorithm"], columns=["k"])
        df_coreset_costs = df_coreset_costs.rename_axis(None, axis=0).rename_axis(None, axis=1)
        
        html_str = ""
        html_str += f'<div style="border:solid 1px #eee; float:left;">'
        html_str += f"<h4 style='text-align:center;'>Real costs</h4>"
        html_str += df_real_costs.to_html()
        html_str += "</div>"
        
        html_str += f'<div style="border:solid 1px #eee; float:left;">'
        html_str += f"<h4 style='text-align:center;'>Coreset costs</h4>"
        html_str += df_coreset_costs.to_html()
        html_str += "</div>"
        display(HTML(html_str))
    
    display(HTML(f'<h4>Distortions</h4>'))
    df_distortions = pd.pivot_table(df_filtered, values="distortion_max", index=["algorithm"], columns=["k"])
    df_distortions = df_distortions.rename_axis(None, axis=0).rename_axis(None, axis=1)
    display(df_distortions)
    
def add_combined_mean_std(df: pd.DataFrame, attr: str):
    def combiner(row) -> str:
        mean = row[f"{attr}_mean"]
        std =  row[f"{attr}_std"]
        return f"{mean:0.2f} ({std:0.3f})"
    df[f"{attr}_mean_std"] = df.apply(lambda x: combiner(x), axis=1)

In [4]:
data_results_dir = Path("../data/experiments-skadi/experiments/")
run_files = list(data_results_dir.glob("**/*.json"))

In [5]:
len(run_files)

9445

In [6]:
run_files_filtered = [f for f in run_files if len(list(f.parent.glob("*.txt"))) > 0]

In [7]:
len(run_files_filtered)

9417

In [8]:
df_cost_data = get_costs(run_files_filtered)

In [9]:
include_datasets = [
    'Caltech', 'Caltech+PCA', 'Census', 'Census+PCA', 'Covertype', 'Covertype+PCA', 
    'Tower', 'NYTimes', 'NYTimes+PCA', 
    # 'Benchmark-1.0', 'Benchmark-1.5', 'Benchmark-2.0',
    'Benchmark',
]
include_algorithms = [
    'StreamKM++', 'Group Sampling', 'Ray Maker', 'BICO', 'Sensitivity Sampling', 
    # 'StreamKM++ (buggy)'
]
df_cost_filtered = df_cost_data[df_cost_data.algorithm.isin(include_algorithms) & df_cost_data.dataset.isin(include_datasets)]
#df_cost_filtered = df_cost_data[df_cost_data.algorithm.isin(include_algorithms)]

In [10]:
df_aggr_costs = aggregate_costs(df_cost_filtered)

In [11]:
colors = sns.color_palette("tab10")
algorithm_colors = {algo: colors[i] for i, algo in enumerate(df_aggr_costs.algorithm.unique())}

In [12]:
df_aggr_costs["k_formatted"] = "$k$=" + df_aggr_costs["k"].astype(str)

In [13]:
df_experiment_counts = pd.pivot_table(
    data=df_aggr_costs,
    values="experiment_count",
    index=["dataset", "k", "coreset_size_factor"],
    columns=["algorithm"],
)

df_experiment_counts

algorithm                          BICO  Group Sampling  Ray Maker  \
dataset   k   coreset_size_factor                                    
Benchmark 10  50                   10.0            10.0       10.0   
              100                  10.0            10.0       10.0   
              200                  10.0            10.0       10.0   
              500                  10.0            10.0       10.0   
          20  50                   10.0            10.0       10.0   
...                                 ...             ...        ...   
Tower     80  500                  10.0            10.0       10.0   
          100 50                   10.0            10.0       10.0   
              100                  10.0            10.0       10.0   
              200                  10.0            10.0       10.0   
              500                  10.0            10.0       10.0   

algorithm                          Sensitivity Sampling  StreamKM++  
dataset   k   coreset_size_factor                                    
Benchmark 10  50                                   10.0        10.0  
              100                                  10.0        10.0  
              200                                  10.0        10.0  
              500                                  10.0         NaN  
          20  50                                   10.0        10.0  
...                                                 ...         ...  
Tower     80  500                                  10.0         NaN  
          100 50                                   10.0        10.0  
              100                                  10.0        10.0  
              200                                  10.0        10.0  
              500                                  10.0         NaN  

[196 rows x 5 columns]

In [14]:
df_experiment_counts.to_excel("experiment-counts.xlsx")

In [15]:
add_combined_mean_std(df=df_aggr_costs, attr="distortion")

df_results_table = pd.pivot_table(
    data=df_aggr_costs,
    values="distortion_mean_std",
    index=["dataset", "k", "coreset_size_factor"],
    columns=["algorithm"],
    aggfunc=lambda x: ' '.join(x)
)

df_results_table

algorithm                                  BICO Group Sampling     Ray Maker  \
dataset   k   coreset_size_factor                                              
Benchmark 10  50                   3.40 (0.440)   1.02 (0.010)  5.05 (0.157)   
              100                  3.24 (0.729)   1.01 (0.004)  3.84 (0.081)   
              200                  2.90 (0.153)   1.01 (0.002)  3.48 (0.052)   
              500                  2.62 (0.095)   1.01 (0.001)  3.40 (0.058)   
          20  50                   3.22 (0.160)   1.04 (0.004)  5.52 (0.266)   
...                                         ...            ...           ...   
Tower     80  500                  1.02 (0.001)   1.02 (0.003)  1.02 (0.001)   
          100 50                   1.19 (0.007)   1.10 (0.009)  1.45 (0.010)   
              100                  1.10 (0.008)   1.06 (0.007)  1.12 (0.005)   
              200                  1.04 (0.001)   1.03 (0.005)  1.05 (0.002)   
              500                  1.01 (0.001)   1.02 (0.003)  1.03 (0.002)   

algorithm                         Sensitivity Sampling    StreamKM++  
dataset   k   coreset_size_factor                                     
Benchmark 10  50                          1.02 (0.005)  1.07 (0.005)  
              100                         1.01 (0.003)  1.05 (0.004)  
              200                         1.01 (0.002)  1.04 (0.002)  
              500                         1.00 (0.001)           NaN  
          20  50                          1.02 (0.003)  1.08 (0.006)  
...                                                ...           ...  
Tower     80  500                         1.01 (0.004)           NaN  
          100 50                          1.03 (0.008)  1.05 (0.002)  
              100                         1.02 (0.007)  1.03 (0.001)  
              200                         1.01 (0.003)  1.02 (0.000)  
              500                         1.01 (0.003)           NaN  

[196 rows x 5 columns]

In [16]:
df_results_table.to_excel("distortions.xlsx")

In [17]:
add_combined_mean_std(df=df_aggr_costs, attr="distortion_synthetic2")

df_results_table = pd.pivot_table(
    data=df_aggr_costs,
    values="distortion_synthetic2_mean_std",
    index=["dataset", "k", "coreset_size_factor"],
    columns=["algorithm"],
    aggfunc=lambda x: ' '.join(x)
)

df_results_table

algorithm                                  BICO Group Sampling     Ray Maker  \
dataset   k   coreset_size_factor                                              
Benchmark 10  50                      nan (nan)      nan (nan)     nan (nan)   
              100                     nan (nan)      nan (nan)     nan (nan)   
              200                     nan (nan)      nan (nan)     nan (nan)   
              500                     nan (nan)      nan (nan)     nan (nan)   
          20  50                      nan (nan)      nan (nan)     nan (nan)   
...                                         ...            ...           ...   
Tower     80  500                  1.00 (0.001)   1.02 (0.005)  1.01 (0.001)   
          100 50                   1.04 (0.008)   1.03 (0.016)  1.26 (0.016)   
              100                  1.02 (0.003)   1.02 (0.007)  1.07 (0.004)   
              200                  1.01 (0.003)   1.02 (0.008)  1.02 (0.003)   
              500                  1.00 (0.001)   1.02 (0.004)  1.01 (0.001)   

algorithm                         Sensitivity Sampling    StreamKM++  
dataset   k   coreset_size_factor                                     
Benchmark 10  50                             nan (nan)     nan (nan)  
              100                            nan (nan)     nan (nan)  
              200                            nan (nan)     nan (nan)  
              500                            nan (nan)           NaN  
          20  50                             nan (nan)     nan (nan)  
...                                                ...           ...  
Tower     80  500                         1.02 (0.008)           NaN  
          100 50                          1.08 (0.021)  1.01 (0.002)  
              100                         1.05 (0.014)  1.01 (0.001)  
              200                         1.03 (0.009)  1.00 (0.001)  
              500                         1.02 (0.004)           NaN  

[196 rows x 5 columns]

In [18]:
df_results_table.to_excel("distortions-convex-clustering.xlsx")

In [19]:
add_combined_mean_std(df=df_aggr_costs, attr="distortion_synthetic")

df_results_table = pd.pivot_table(
    data=df_aggr_costs,
    values="distortion_synthetic_mean_std",
    index=["dataset", "k", "coreset_size_factor"],
    columns=["algorithm"],
    aggfunc=lambda x: ' '.join(x)
)
df_results_table.to_excel("distortions-meb-clustering.xlsx")
df_results_table

algorithm                                  BICO Group Sampling     Ray Maker  \
dataset   k   coreset_size_factor                                              
Benchmark 10  50                      nan (nan)      nan (nan)     nan (nan)   
              100                     nan (nan)      nan (nan)     nan (nan)   
              200                     nan (nan)      nan (nan)     nan (nan)   
              500                     nan (nan)      nan (nan)     nan (nan)   
          20  50                      nan (nan)      nan (nan)     nan (nan)   
...                                         ...            ...           ...   
Tower     80  500                  1.00 (0.000)   1.03 (0.001)  1.00 (0.000)   
          100 50                   1.00 (0.000)   1.03 (0.007)  1.19 (0.015)   
              100                  1.00 (0.000)   1.03 (0.006)  1.05 (0.004)   
              200                  1.00 (0.000)   1.03 (0.003)  1.02 (0.002)   
              500                  1.00 (0.000)   1.03 (0.002)  1.00 (0.000)   

algorithm                         Sensitivity Sampling    StreamKM++  
dataset   k   coreset_size_factor                                     
Benchmark 10  50                             nan (nan)     nan (nan)  
              100                            nan (nan)     nan (nan)  
              200                            nan (nan)     nan (nan)  
              500                            nan (nan)           NaN  
          20  50                             nan (nan)     nan (nan)  
...                                                ...           ...  
Tower     80  500                         1.01 (0.006)           NaN  
          100 50                          1.06 (0.012)  1.00 (0.000)  
              100                         1.04 (0.011)  1.00 (0.000)  
              200                         1.03 (0.007)  1.00 (0.000)  
              500                         1.01 (0.003)           NaN  

[196 rows x 5 columns]

In [20]:
add_combined_mean_std(df=df_aggr_costs, attr="running_time")

df_results_table = pd.pivot_table(
    data=df_aggr_costs,
    values="running_time_mean_std",
    index=["dataset", "k", "coreset_size_factor"],
    columns=["algorithm"],
    aggfunc=lambda x: ' '.join(x)
)

df_results_table

algorithm                                    BICO    Group Sampling  \
dataset   k   coreset_size_factor                                     
Benchmark 10  50                    18.60 (2.675)   117.10 (10.203)   
              100                   20.50 (3.923)    119.90 (7.233)   
              200                   25.40 (3.373)   118.30 (10.275)   
              500                   31.40 (7.106)    124.70 (9.440)   
          20  50                   104.30 (6.993)  1015.90 (74.061)   
...                                           ...               ...   
Tower     80  500                   20.10 (4.175)  1122.40 (13.656)   
          100 50                     6.90 (1.792)  1419.10 (22.083)   
              100                    7.50 (1.354)  1434.20 (21.343)   
              200                   10.50 (0.850)  1427.80 (14.559)   
              500                   28.50 (7.200)   1444.20 (8.892)   

algorithm                                Ray Maker Sensitivity Sampling  \
dataset   k   coreset_size_factor                                         
Benchmark 10  50                   138.40 (14.871)       106.10 (9.848)   
              100                  142.60 (17.392)       107.20 (8.121)   
              200                  150.80 (19.315)       107.90 (7.852)   
              500                   187.80 (9.852)      104.90 (14.372)   
          20  50                   904.10 (68.385)      932.90 (81.004)   
...                                            ...                  ...   
Tower     80  500                  339.70 (10.874)       269.20 (5.073)   
          100 50                    174.80 (6.052)       324.50 (7.990)   
              100                   197.90 (3.247)       327.00 (7.498)   
              200                   240.60 (6.569)       330.50 (7.091)   
              500                   383.80 (9.886)       338.00 (4.546)   

algorithm                                  StreamKM++  
dataset   k   coreset_size_factor                      
Benchmark 10  50                      736.50 (60.184)  
              100                    1460.40 (86.997)  
              200                   2889.30 (138.969)  
              500                                 NaN  
          20  50                    7233.10 (555.451)  
...                                               ...  
Tower     80  500                                 NaN  
          100 50                     3402.70 (35.512)  
              100                   6859.20 (120.374)  
              200                  13530.20 (474.776)  
              500                                 NaN  

[196 rows x 5 columns]

In [21]:
df_results_table.to_excel("running-times.xlsx")

In [22]:
df_results_table = pd.pivot_table(
    data=df_aggr_costs,
    values="distortion_mean_std",
    index=["dataset", "k", "coreset_size_factor"],
    columns=["algorithm"],
    aggfunc=lambda x: ' '.join(x)
)

df_results_table

algorithm                                  BICO Group Sampling     Ray Maker  \
dataset   k   coreset_size_factor                                              
Benchmark 10  50                   3.40 (0.440)   1.02 (0.010)  5.05 (0.157)   
              100                  3.24 (0.729)   1.01 (0.004)  3.84 (0.081)   
              200                  2.90 (0.153)   1.01 (0.002)  3.48 (0.052)   
              500                  2.62 (0.095)   1.01 (0.001)  3.40 (0.058)   
          20  50                   3.22 (0.160)   1.04 (0.004)  5.52 (0.266)   
...                                         ...            ...           ...   
Tower     80  500                  1.02 (0.001)   1.02 (0.003)  1.02 (0.001)   
          100 50                   1.19 (0.007)   1.10 (0.009)  1.45 (0.010)   
              100                  1.10 (0.008)   1.06 (0.007)  1.12 (0.005)   
              200                  1.04 (0.001)   1.03 (0.005)  1.05 (0.002)   
              500                  1.01 (0.001)   1.02 (0.003)  1.03 (0.002)   

algorithm                         Sensitivity Sampling    StreamKM++  
dataset   k   coreset_size_factor                                     
Benchmark 10  50                          1.02 (0.005)  1.07 (0.005)  
              100                         1.01 (0.003)  1.05 (0.004)  
              200                         1.01 (0.002)  1.04 (0.002)  
              500                         1.00 (0.001)           NaN  
          20  50                          1.02 (0.003)  1.08 (0.006)  
...                                                ...           ...  
Tower     80  500                         1.01 (0.004)           NaN  
          100 50                          1.03 (0.008)  1.05 (0.002)  
              100                         1.02 (0.007)  1.03 (0.001)  
              200                         1.01 (0.003)  1.02 (0.000)  
              500                         1.01 (0.003)           NaN  

[196 rows x 5 columns]